# YOLOv8 Classification: Train + Validate + Report (Global Python)
> ใช้กับ `ultralytics` รุ่นที่รองรับ `yolov8s-cls.pt`  
> โครงสร้าง dataset แบบแยกโฟลเดอร์:  
```
dataset/
  train/
    classA/ img1.jpg ...
    classB/ img2.jpg ...
  val/
    classA/ ...
    classB/ ...
  test/
    classA/ ...
    classB/ ...
```


In [ ]:
# ติดตั้งไลบรารีที่จำเป็น (ใน Python global ของคุณ)
!python -m pip install --upgrade pip
!python -m pip install ultralytics==8.2.103 matplotlib pandas scikit-learn

# สำหรับ PyTorch: เลือก wheel ให้ตรงกับ CUDA ของคุณ (12.1 หรือ 12.4)
# ตัวอย่างติดตั้ง CUDA 12.4:
# !python -m pip install --index-url https://download.pytorch.org/whl/cu124 torch torchvision
# หรือ CUDA 12.1:
# !python -m pip install --index-url https://download.pytorch.org/whl/cu121 torch torchvision


In [ ]:
import sys, platform
import torch
import matplotlib, pandas
from ultralytics import YOLO

print("Python:", sys.version)
print("Platform:", platform.platform())
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("Matplotlib:", matplotlib.__version__)
print("Pandas:", pandas.__version__)


In [ ]:
\
from pathlib import Path

# ===== ปรับพาธตามเครื่องคุณ =====
PROJECT_DIR = Path(r"D:\herbSkin_Project66")
DATASET_DIR = PROJECT_DIR / r"Dataset\herb2"   # ต้องมี train/val/test
MODEL_WEIGHTS = "yolov8s-cls.pt"              # โมเดลสำหรับ classification

# Training params
EPOCHS = 50
BATCH  = 32
IMGSZ  = 224
WORKERS = 4

print("PROJECT_DIR:", PROJECT_DIR)
print("DATASET_DIR:", DATASET_DIR)


In [ ]:
# ตรวจสอบโครงสร้าง dataset และรายชื่อคลาส
import os

def list_classes(split_dir: Path):
    classes = []
    if split_dir.exists():
        for p in sorted([d for d in split_dir.iterdir() if d.is_dir()]):
            classes.append(p.name)
    return classes

train_dir = DATASET_DIR / "train"
val_dir   = DATASET_DIR / "val"
test_dir  = DATASET_DIR / "test"

print("Exist(train):", train_dir.exists(), " Exist(val):", val_dir.exists(), " Exist(test):", test_dir.exists())
print("Classes(train):", list_classes(train_dir))
print("Classes(val):", list_classes(val_dir))
print("Classes(test):", list_classes(test_dir))


In [ ]:
# เทรนโมเดล YOLOv8 Classification
from ultralytics import YOLO

model = YOLO(MODEL_WEIGHTS)  # yolov8s-cls.pt

results = model.train(
    data=str(DATASET_DIR),
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    workers=WORKERS,
    patience=20,
    lr0=0.001,
    optimizer='AdamW',
    verbose=True
)
results


In [ ]:
# ประเมินผลบน val (หรือ test)
val_metrics = model.val(split='val', imgsz=IMGSZ, batch=BATCH, workers=WORKERS)
val_metrics


In [ ]:
# สร้าง Confusion Matrix จากชุด val
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

class_names = sorted([d.name for d in (DATASET_DIR / "val").iterdir() if d.is_dir()])
class_to_idx = {c:i for i, c in enumerate(class_names)}

image_paths = []
y_true = []
for c in class_names:
    cls_dir = DATASET_DIR / "val" / c
    for img in cls_dir.rglob("*"):
        if img.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".webp"}:
            image_paths.append(str(img))
            y_true.append(class_to_idx[c])

preds = []
batch_size = 64
for i in range(0, len(image_paths), batch_size):
    batch = image_paths[i:i+batch_size]
    res = model.predict(batch, imgsz=IMGSZ, verbose=False)
    for r in res:
        preds.append(int(r.probs.top1))

y_true = np.array(y_true)
y_pred = np.array(preds)

cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
plt.figure(figsize=(8,8))
disp.plot(xticks_rotation=45, values_format='d')
plt.title("Confusion Matrix (val)")
plt.tight_layout()
plt.show()


In [ ]:
# ตัวอย่างการพยากรณ์ภาพเดี่ยว/โฟลเดอร์
sample = str(next((DATASET_DIR / "test").rglob("*.*")))  # หยิบไฟล์แรกใน test เป็นตัวอย่าง
print("Sample:", sample)
res = model.predict(sample, imgsz=IMGSZ)
print(res[0].probs)


In [ ]:
# Export โมเดล (ถ้าต้องการ)
# ตัวเลือก: onnx, openvino, coreml, engine(TensorRT), torchscript ฯลฯ
# ตัวอย่าง:
# model.export(format="onnx", imgsz=IMGSZ, opset=12)


## Tips
- หากต้องการ **resume training**: ใส่ `resume=True` ใน `model.train(...)`
- หากชุดข้อมูลไม่สมดุล ลองปรับ `lr0`, `batch` ให้เหมาะสม
- สามารถใช้ global Python ได้ ไม่จำเป็นต้องสร้าง venv
